# This is a second order wave equation with Dirichlet boundary conditions used to test different minimizers. 

We just implement the wave equation as a toy model to try different minimizers and strategies. 

Let's solve this 1-dimensional wave equation:

```math
\begin{align*}
∂^2_t u(x, t) = c^2 ∂^2_x u(x, t) \quad & \textsf{for all } 0 < x < 1 \text{ and } t > 0 \, , \\
u(0, t) = u(1, t) = 0 \quad & \textsf{for all } t > 0 \, , \\
u(x, 0) = x (1-x)     \quad & \textsf{for all } 0 < x < 1 \, , \\
∂_t u(x, 0) = 0       \quad & \textsf{for all } 0 < x < 1 \, , \\
\end{align*}
```

with grid discretization `dx = 0.1` and physics-informed neural networks.

Further, the solution of this equation with the given boundary conditions is presented.


In [ ]:
using NeuralPDE, Lux, LuxCUDA, Optimization, OptimizationOptimJL
using ModelingToolkit, IntervalSets, Plots, JLD2
using ComponentArrays, Random
const gpud = gpu_device()

@parameters t, x
@variables u(..)
Dxx = Differential(x)^2
Dtt = Differential(t)^2
Dt = Differential(t)

#2D PDE
C = 1
eq = Dtt(u(t, x)) ~ C^2 * Dxx(u(t, x))

# Initial and boundary conditions
bcs = [u(t, 0) ~ 0.0,# for all t > 0
    u(t, 1) ~ 0.0,# for all t > 0
    u(0, x) ~ x * (1.0 - x), #for all 0 < x < 1
    Dt(u(0, x)) ~ 0.0] #for all  0 < x < 1]

# Space and time domains
ti = 0.0
tf = 1.0
xi = 0.0
xf = 1.0
domains = [t ∈ Interval(ti, tf),
    x ∈ Interval(xi, xf)]
# Discretization
dx = 0.1

@named wave = PDESystem(eq, bcs, domains, [t, x], [u(t, x)])

# Neural network
chain = Chain(Dense(2, 16, σ), Dense(16, 16, σ), Dense(16, 1))

ps = Lux.setup(Random.default_rng(), chain)[1]
ps = ps |> ComponentArray |> gpud .|> Float64

strategy = QuasiRandomTraining(100)
#strategy = QuadratureTraining(; abstol = 1e-6, reltol = 1e-6, batch = 200)
#strategy = GridTraining(0.05)
discretization = PhysicsInformedNN(chain, strategy; init_params = ps#, param_estim = true
#, additional_loss = additional_loss
)

prob = discretize(wave, discretization)
sym_prob = symbolic_discretize(wave, discretization)

pde_inner_loss_functions = sym_prob.loss_functions.pde_loss_functions
bcs_inner_loss_functions = sym_prob.loss_functions.bc_loss_functions

loss = Float64[]
callback = function (p, l)
    push!(loss, l)
    println("loss: ", l)
    println("pde_losses: ", map(l_ -> l_(p.u), pde_inner_loss_functions))
    println("bcs_losses: ", map(l_ -> l_(p.u), bcs_inner_loss_functions))
    return false
end
#discretization = PhysicsInformedNN(chain, GridTraining(dx))

@named pde_system = PDESystem(eq, bcs, domains, [t, x], [u(t, x)])
prob = discretize(pde_system, discretization)



In [ ]:

opt = OptimizationOptimJL.BFGS()
res = Optimization.solve(prob, opt; callback, maxiters = 12000)
phi = discretization.phi

In [ ]:
using Plots

plot(loss, yscale=:log10, xlabel="Iteration", ylabel="Loss", title="Loss vs Iteration"  )
#savefig("loss_pure_2_wave_BFGS.png")

In [ ]:
#using Plots

ts = ti:dx:tf
xs = xi:dx:xf
function analytic_sol_func(t, x)
    sum([(8 / (k^3 * pi^3)) * sin(k * pi * x) * cos(C * k * pi * t) for k in 1:2:50000])
end

u_predict = [first(Array(phi([t, x], res.u))) for t in ts for x in xs]
u_real = [analytic_sol_func(t, x) for t in ts for x in xs]

diff_u = abs.(u_predict .- u_real)
p1 = plot(ts, xs, u_real, linetype = :contourf, title = "analytic");
p2 = plot(ts, xs, u_predict, linetype = :contourf, title = "predict");
p3 = plot(ts, xs, diff_u, linetype = :contourf, title = "error");
plot(p1, p2, p3)
